In [1]:
### NOTE: ALL @APP.ROUTE LINES ARE COMMENTED OUT FOR JUPYTER NOTEBOOK ONLY. TO BE ADDED BACK IN APP.PY FILE.

import os

import pandas as pd
import numpy as np

import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine

from flask import Flask, jsonify, render_template
from flask_sqlalchemy import SQLAlchemy

app = Flask(__name__)

In [2]:
# Database Setup
engine = create_engine("sqlite:///db/gunviolence_db.sqlite")
df_gunviolence = pd.read_sql('select * from gunviolence_db', engine)
print(len(df_gunviolence))

425


In [3]:
#@app.route("/")
def index():
    """Return the homepage."""
    return render_template("index.html")

In [11]:
#@app.route("/states")
def list_states():
    """Return a list of states."""

    # Use Pandas to perform the sql query
    engine = create_engine("sqlite:///db/gunviolence_db.sqlite")
    df_gunviolence_states = pd.read_sql('select State from gunviolence_db', engine)
    
    stmt = list(df_gunviolence_states.sort_values(by=['State'])['State'].unique())

    # Return a list of the column names (state names)
    return jsonify(stmt)
    return stmt

states = list_states()
print(states)

['Alabama', 'Arizona', 'Arkansas', 'California', 'Colorado', 'Connecticut', 'Delaware', 'District of Columbia', 'Florida', 'Georgia', 'Illinois', 'Indiana', 'Iowa', 'Kansas', 'Kentucky', 'Louisiana', 'Maine', 'Maryland', 'Massachusetts', 'Michigan', 'Minnesota', 'Mississippi', 'Missouri', 'Montana', 'Nebraska', 'Nevada', 'New Hampshire', 'New Jersey', 'New Mexico', 'New York', 'North Carolina', 'North Dakota', 'Ohio', 'Oklahoma', 'Oregon', 'Pennsylvania', 'Rhode Island', 'South Carolina', 'Tennessee', 'Texas', 'Utah', 'Vermont', 'Virginia', 'Washington', 'West Virginia', 'Wisconsin']


In [76]:
# METADATA TABLE: Create a summary of gun violence by state. 
# Changed df_gun_violence2 -> df_gun_violence_meta

#@app.route("/metadata/<state>")
def gunviolence_metadata(state):
    """Return the gun violence data summary (metadata) for a given state."""

    engine = create_engine("sqlite:///db/gunviolence_db.sqlite")
    df_gunviolence_metadata = pd.read_sql(f"select * from gunviolence_db where State = '{state}' ", engine)
    df_gunviolence_metadata["Killed"] = df_gunviolence_metadata["Killed"].astype(int)
    df_gunviolence_metadata["Injured"] = df_gunviolence_metadata["Injured"].astype(int)
            
    # Create a dictionary entry for each row of gun violence summary data (metadata)
    gunviolence_metadata = {
        "Incident_ID": df_gunviolence_metadata['Incident ID'].value_counts().sum(), 
        "Number_Killed": df_gunviolence_metadata[df_gunviolence_metadata["Killed"] > 0]["Killed"].value_counts().sum(),
        "Number_Injured": df_gunviolence_metadata[df_gunviolence_metadata["Injured"] > 0]["Injured"].value_counts().sum()
    }
    
    print(gunviolence_metadata)
    return jsonify(gunviolence_metadata)

gunviolence_metadata('New Jersey')

{'Incident_ID': 15, 'Number_Killed': 0, 'Number_Injured': 7}


RuntimeError: Working outside of application context.

This typically means that you attempted to use functionality that needed
to interface with the current application object in some way. To solve
this, set up an application context with app.app_context().  See the
documentation for more information.

In [ ]:
# GOAL: Create a dictionary entry for each row of gun violence summary data (metadata)
# { 'year' : 2018
#   'incident_count': "IncidentID".count,
#   'number_killed': "Killed".sum,
#   'number_injured': "Injured".sum
# }

In [98]:
# Create app route to pull all data points for charts.
#@app.route("/states/<state>")
def states(state):
    """Return `incident_ids`, `incident_dates`, `city_or_county`, `number_killed` and `number_injured`."""

    # Filter the data based on the state
    df_gunviolence_bystate = pd.read_sql(f"select * from gunviolence_db where State = '{state}' ", engine)

    # Format the data to send as json
    data = {
        "incident_ids": df_gunviolence_bystate["Incident ID"].tolist(),
        "incident_dates": df_gunviolence_bystate["Incident Date"].tolist(),
        "cities_or_counties": df_gunviolence_bystate["City Or County"].tolist(),
        "number_killed": df_gunviolence_bystate["Killed"].tolist(),
        "number_injured": df_gunviolence_bystate["Injured"].tolist()
    }
    #return jsonify(data)
    return(data) 

states('Wisconsin')

{'incident_ids': ['1289543', '1300872', '1290535', '1289545', '1292121'],
 'incident_dates': ['December 31',
  'December 31',
  'December 31',
  'December 31',
  'December 31'],
 'cities_or_counties': ['Milwaukee',
  'Milwaukee',
  'Oshkosh',
  'Milwaukee',
  'Racine'],
 'number_killed': ['1', '0', '0', '1', '0'],
 'number_injured': ['0', '0', '0', '0', '1']}